# ROM - Run offline experiments

#### Description

The purpose of this notebook is to run ROM offline experiments. 

In [18]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
from rom import ROMTrainer, ROMTrainerConfig
from utils import *
from modelling_pkg.config import *
import pandas as pd
import warnings

warnings.filterwarnings(
    "ignore", message="Skipping features without any observed values"
)

pd.set_option("display.max_columns", None)


DATADIR = "../../../" + DATADIR

dd# Load data

In [20]:
countries_dir = f"{DATADIR}"
# countries_dir = DATADIR_RTP + '/countries'

# counts = pd.read_csv(countries_dir+'/0_counts.csv')
# counts

<div class="alert alert-block alert-warning"><b>Select countries for training</b></div>

In [4]:
# countries = ['DENMARK', 'FINLAND', 'HONG_KONG_SAR,_CHINA', 'NETHERLANDS', 'JAPAN', 'MEXICO']
# countries = ['ARGENTINA', 'BRAZIL', 'CHILE', 'COLUMBIA', 'CROATIA', 'CZECHIA']
# countries = ['UNITED_STATES_OF_AMERICA']

# poc
countries = [
    "Colombia",
    "Czech_Republic",
    "Italy",
    "Taiwan",
    "Thailand",
    "United_Arab_Emirates",
]

# spans = defaultdict(lambda: ('2023-12-01', '2024-04-01', '2024-04-01', '2024-06-01'))
# spans = ('2023-12-01', '2024-04-01', '2024-04-01', '2024-06-01')
spans = ("2023-10-01", "2024-04-01", "2024-04-01", "2024-07-01")
# all countries:
# countries = [
#     'ARGENTINA', 'AUSTRALIA', 'AUSTRIA', 'BELGIUM', 'BRAZIL', 'BULGARIA', 'CHILE', 'COLUMBIA', 'CROATIA', 'CZECHIA', 'DENMARK',
#     'ECUADOR', 'EGYPT', 'FINLAND', 'FRANCE', 'GERMANY', 'GREECE', 'HONG_KONG_SAR', 'HUNGARY', 'IRELAND', 'ISRAEL', 'ITALY', 'MALAYSIA',
#     'NETHERLANDS', 'NEW_ZEALAND', 'NORWAY', 'PHILIPPINES,_THE', 'POLAND', 'PORTUGAL', 'RUSSIA', 'SAUDI_ARABIA', 'SOUTH_AFRICA',
#     'SPAIN', 'SWEDEN', 'SWITZERLAND', 'TAIWAN', 'THAILAND', 'UNITED_ARAB_EMIRATES', 'UNITED_KINGDOM', 'UNITED_STATES_OF_AMERICA', 'VIETNAM',
# ]

In [6]:
data_dict = {}
for c in countries:
    data = pd.read_csv(
        f"{countries_dir}/{c}.csv",
        parse_dates=["created_date_off", "updated_date_off"],
        # index_col=0,
        low_memory=False,
    )
    data["accepted"] = data.final_outcome.apply(map_final_outcome)

    for col in num_cols:
        if col not in data.columns:
            data[col] = 0

    data_dict[c] = data.dropna(subset="accepted")
list(map(lambda x: (x[0], len(x[1])), data_dict.items()))

[('Colombia', 10605),
 ('Czech_Republic', 21160),
 ('Italy', 136580),
 ('Taiwan', 25919),
 ('Thailand', 23188),
 ('United_Arab_Emirates', 14197)]

In [7]:
for c in countries:
    print(c)
    # split_trn_tst(data_dict[c], c, [bin_cols, cat_cols, num_cols], spans=('2022-12-01', '2023-06-01', '2023-06-01', '2023-09-01'), verbose=True)
    split_trn_tst(
        data_dict[c], c, [bin_cols, cat_cols, num_cols], spans=spans, verbose=True
    )
    print()

Colombia
('2023-10-01', '2024-04-01', '2024-04-01', '2024-07-01')
Train data shape: (2644, 137)
Test data shape: (1301, 137)

Czech_Republic
('2023-10-01', '2024-04-01', '2024-04-01', '2024-07-01')
Train data shape: (5836, 138)
Test data shape: (3282, 138)

Italy
('2023-10-01', '2024-04-01', '2024-04-01', '2024-07-01')
Train data shape: (30458, 138)
Test data shape: (13630, 138)

Taiwan
('2023-10-01', '2024-04-01', '2024-04-01', '2024-07-01')
Train data shape: (7454, 136)
Test data shape: (3400, 136)

Thailand
('2023-10-01', '2024-04-01', '2024-04-01', '2024-07-01')
Train data shape: (7062, 136)
Test data shape: (2887, 136)

United_Arab_Emirates
('2023-10-01', '2024-04-01', '2024-04-01', '2024-07-01')
Train data shape: (5170, 137)
Test data shape: (1545, 137)


# Configuration

In [8]:
N_TRIALS = 3
timeout_minutes = 2
CM = 1
SAVE_IMMEDIATELY_TO = f"{DATADIR}/rom_results"
# SAVE_IMMEDIATELY_TO = None
# EXPERIMENT = f'augm-basic-vs-shift-01'
# EXPERIMENT = f'augm-clip_min-01'
# EXPERIMENT = f'n_trials,init_param'
# todo if experiment exists for the countries, raise warning

In [9]:
EXPERIMENT = "v07-test"
rom_configs = [
    ROMTrainerConfig(
        experiment=EXPERIMENT,
        name=f"{a:02},{b}",
        n_trials=N_TRIALS,
        timeout_minutes=timeout_minutes,
        augm_variant="basic",
        augm_basic_params=(a, b),
        clip_min=CM,
        spans=spans,
    )
    for a, b in sorted([(8, 26), (10, 37), (11, 19), (14, 28), (10, 43)])
]

rom_configs

[ROMTrainerConfig(experiment='v07-test', name='08,26', estimator='xgbc', augm_variant='basic', gammas=(4, 6, 8), sample=None, n_trials=3, timeout_minutes=2, augm_basic_params=(8, 26), augm_basic_weight=1, augm_basic_weight_asym=False, augm__rshift_sigma=2, xgbc__initial_params=True, optuna__seed=0, random_state=0, clip_min=1, spans=('2023-10-01', '2024-04-01', '2024-04-01', '2024-07-01')),
 ROMTrainerConfig(experiment='v07-test', name='10,37', estimator='xgbc', augm_variant='basic', gammas=(4, 6, 8), sample=None, n_trials=3, timeout_minutes=2, augm_basic_params=(10, 37), augm_basic_weight=1, augm_basic_weight_asym=False, augm__rshift_sigma=2, xgbc__initial_params=True, optuna__seed=0, random_state=0, clip_min=1, spans=('2023-10-01', '2024-04-01', '2024-04-01', '2024-07-01')),
 ROMTrainerConfig(experiment='v07-test', name='10,43', estimator='xgbc', augm_variant='basic', gammas=(4, 6, 8), sample=None, n_trials=3, timeout_minutes=2, augm_basic_params=(10, 43), augm_basic_weight=1, augm_ba

# Train

In [10]:
rom_configs[0]

ROMTrainerConfig(experiment='v07-test', name='08,26', estimator='xgbc', augm_variant='basic', gammas=(4, 6, 8), sample=None, n_trials=3, timeout_minutes=2, augm_basic_params=(8, 26), augm_basic_weight=1, augm_basic_weight_asym=False, augm__rshift_sigma=2, xgbc__initial_params=True, optuna__seed=0, random_state=0, clip_min=1, spans=('2023-10-01', '2024-04-01', '2024-04-01', '2024-07-01'))

In [11]:
for c in num_cols:
    display(data_dict[countries[0]][c].isna().value_counts())

False    10605
Name: log_published_revenue, dtype: int64

False    10605
Name: log_published_revenue_sq, dtype: int64

False    10168
True       437
Name: log_potential_revenue, dtype: int64

False    10605
Name: log_offer_published_revenue, dtype: int64

False    10604
True         1
Name: log_shipments, dtype: int64

...

False    10605
Name: PROD.MIX:DDIMPORT, dtype: int64

False    10605
Name: PROD.MIX:DD3RD, dtype: int64

False    10605
Name: PROD.MIX:TDDOM, dtype: int64

False    10605
Name: PROD.MIX:TD3DOM, dtype: int64

False    10605
Name: PROD.MIX:OTHER, dtype: int64

In [15]:
# trainer = ROMTrainer(bin_cols, cat_cols, num_cols)
# trainer._cfg = rom_configs[0]
# n_features = trainer._temp_prep_pipe.fit(data_dict[countries[0]]).transform(data_dict[countries[0]].sample(1)).shape[1]
# test, _ = trainer._train_model(data_dict[countries[0]], data_dict[countries[0]]['accepted'], n_features, rom_configs[0], [bin_cols, cat_cols, num_cols], [1] * len(data_dict[countries[0]]))

In [14]:
printmd(f"**{EXPERIMENT}**\n\nCountries: {', '.join(countries)}")

trainer = ROMTrainer(bin_cols, cat_cols, num_cols)

trainer.train_models(
    data_dict, countries, rom_configs, save_immediately_to=SAVE_IMMEDIATELY_TO
)

**v07-test**

Countries: Colombia, Czech_Republic, Italy, Taiwan, Thailand, United_Arab_Emirates



**^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ Colombia (1/6) - 08,26 (1/5) ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^**



('2023-10-01', '2024-04-01', '2024-04-01', '2024-07-01')
Train data shape: (2626, 137)
Test data shape: (1287, 137)


[I 2025-01-28 16:14:02,195] A new study created in memory with name: no-name-16632687-1cd4-405b-9fad-251427054026


before/after aug (2626, 137) (7771, 137)


...

before/after aug (5047, 137) (13466, 137)


  0%|          | 0/3 [00:00<?, ?it/s]

[I 2025-01-28 16:44:41,875] Trial 0 finished with value: 0.2692714976983481 and parameters: {'learning_rate': 0.01087080185549562, 'max_depth': 4, 'n_estimators': 729, 'gamma': 0.01123506477003016}. Best is trial 0 with value: 0.2692714976983481.
[I 2025-01-28 16:44:50,243] Trial 1 finished with value: 0.26121016453748946 and parameters: {'learning_rate': 0.02230934569621654, 'max_depth': 6, 'n_estimators': 721, 'gamma': 0.05498659253091755}. Best is trial 1 with value: 0.26121016453748946.
[I 2025-01-28 16:44:56,671] Trial 2 finished with value: 0.26304746623470815 and parameters: {'learning_rate': 0.03538453590906552, 'max_depth': 5, 'n_estimators': 643, 'gamma': 0.1229607110732571}. Best is trial 1 with value: 0.26121016453748946.
Saving results to '../../../Data/training-data-POC-countries/rom_results/United_Arab_Emirates~2024-04~2024-07/v07-test/14,28'
United_Arab_Emirates


  0%|          | 0/1523 [00:00<?, ?it/s]

,real_discount,pub_rev,accepted,real_rev,rom_discount,rom_pa,rom_exp_rev,rom_discount_4_low,rom_discount_4_mid,rom_discount_4_high,rom_discount_6_low,rom_discount_6_mid,rom_discount_6_high,rom_discount_8_low,rom_discount_8_mid,rom_discount_8_high,rom_diff
18252,64.870329,3116.34,1,1094.76,46,0.718189,0.387822,44,52.0,60,44,52.5,61,44,53.0,62,-18.870329
18253,79.220232,31877.16,0,0.00,90,0.855677,0.085568,90,90.0,90,90,90.0,90,90,90.0,90,10.779768
18254,82.855584,45075.90,0,0.00,90,0.871594,0.087159,90,90.0,90,90,90.0,90,90,90.0,90,7.144416
18255,74.513822,13252.36,0,0.00,78,0.474732,0.104441,78,78.0,78,78,78.5,79,78,84.0,90,3.486178
18256,82.413669,18599.90,1,3271.04,65,0.883013,0.309054,65,66.0,67,65,66.0,67,65,66.5,68,-17.413669
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20832,63.369910,2277.39,1,834.21,60,0.958405,0.383362,60,60.5,61,60,61.0,62,58,60.5,63,-3.369910
20843,63.236068,5130.30,1,1886.10,60,0.915911,0.366365,60,61.0,62,60,61.5,63,60,62.5,65,-3.236068
20845,69.370942,7756.36,1,2375.70,65,0.924838,0.323693,60,63.0,66,60,63.5,67,60,63.5,67,-4.370942
20853,63.835055,17284.50,1,6250.93,65,0.897990,0.314297,65,66.0,67,65,66.0,67,65,66.5,68,1.164945


In [16]:
EXPERIMENT = "vbaseline-test"
rom_configs = [
    ROMTrainerConfig(
        experiment=EXPERIMENT,
        name="no_augmentation",
        n_trials=N_TRIALS,
        timeout_minutes=timeout_minutes,
        augm_variant="basic",
        augm_basic_params=(0, 0),
        clip_min=CM,
        spans=spans,
    )
]

In [17]:
rom_configs[0].augm_basic_weight

1

In [ ]:
printmd(f"**{EXPERIMENT}**\n\nCountries: {', '.join(countries)}")

trainer = ROMTrainer(bin_cols, cat_cols, num_cols)

trainer.train_models(
    data_dict, countries, rom_configs, save_immediately_to=SAVE_IMMEDIATELY_TO
)